In [ ]:
# Install all required libraries
!pip install unbabel-comet
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas sacrebleu sentencepiece

from huggingface_hub import login

# Paste your token here
login(token='hugging face token')

# Download NLTK data needed for BLEU and ROUGE
import nltk
nltk.download('punkt')

# Step 4: Imports
import time
import torch
import psutil
import logging
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
import evaluate

logging.getLogger("transformers").setLevel(logging.ERROR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 5: Paragraph dataset
paragraph_data = [
    {"lang": "ar", "name": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "es", "name": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "fr", "name": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ru", "name": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "ro", "name": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "bg", "name": "Bulgarian", "src": "Кандидатствам за туристическа виза за посещение на Съединените щати. Възнамерявам да остана две седмици и да посетя популярни забележителности като Статуята на свободата и Таймс Скуеър. Ще се върна в родината си след ваканцията.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "cs", "name": "Czech", "src": "Žádám o turistické vízum pro návštěvu Spojených států. Mám v úmyslu zůstat dva týdny a navštívit známá místa jako Sochu svobody a Times Square. Po dovolené se vrátím do své domovské země.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "da", "name": "Danish", "src": "Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "de", "name": "German", "src": "Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"lang": "et", "name": "Estonian", "src": "Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]


# Step 6: Load model and tokenizer
model_name = "facebook/m2m100_1.2B"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

# Step 7: Load evaluation modules
bleu_mod      = evaluate.load("bleu")
bertscore_mod = evaluate.load("bertscore")
meteor_mod    = evaluate.load("meteor")
rouge_mod     = evaluate.load("rouge")
ter_mod       = evaluate.load("ter")
comet_mod     = evaluate.load("comet")

# Step 8: System monitor helper
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Step 9: Translate with M2M100
def translate_m2m100(text: str, src_lang: str, tgt_lang: str = "en"):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt").to(device)
    start = time.perf_counter()
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang)
    )
    mid = time.perf_counter()
    pred = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    end = time.perf_counter()
    return pred, mid - start, end - start, len(tokenizer.tokenize(text))

# Step 10: Evaluate all
for item in paragraph_data:
    src_lang, name, src, ref = item["lang"], item["name"], item["src"], item["ref"]
    print(f"\n=== {name} → English ===")

    ram_before, cpu_before = measure_system()
    pred, t_first, t_total, token_count = translate_m2m100(src, src_lang)
    ram_after, cpu_after = measure_system()

    # Accuracy metrics
    bleu_score   = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res     = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score  = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score    = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res    = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score  = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Output
    print(f"Source    : {src}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}\n")
    print(f"BLEU      : {bleu_score:.4f}")
    print(f"BERTScore : {bert_res['f1'][0]:.4f}")
    print(f"METEOR    : {meteor_score:.4f}")
    print(f"ROUGE-L   : {rouge_score:.4f}")
    print(f"TER       : {ter_score:.4f}")
    print(f"COMET     : {comet_score:.4f}\n")
    print(f"Time to 1st tok : {t_first:.4f}s")
    print(f"Total inf time  : {t_total:.4f}s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}")
    if device.type == "cuda":
        print(f"VRAM used       : {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
    print(f"RAM   : {ram_after - ram_before:.2f} MB")
    print(f"CPU   : {cpu_after - cpu_before:.2f} %")
    print(f"Model size : {sum(p.numel() for p in model.parameters()) * 4 / 1024 ** 2:.2f} MB")
    print("=" * 60)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cpu


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`


...

Source    : Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I apply for a tourist visa to visit the United States. I intend to stay for two weeks and visit famous landmarks such as the Statue of Liberty and Times Square.

BLEU      : 0.5537
BERTScore : 0.9679
METEOR    : 0.6821
ROUGE-L   : 0.7500
TER       : 36.5854
COMET     : 0.8824

Time to 1st tok : 512.2086s
Total inf time  : 512.2230s
Tokens/sec      : 0.10
RAM   : 17.07 MB
CPU   : -3.30 %
Model size : 4728.20 MB

=== German → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I apply for a tourist visa to visit the United States. I intend to stay for two weeks and visit well-known sights such as the Statue of Liberty and Times Square.

BLEU      : 0.5489
BERTScore : 0.9663
METEOR    : 0.6621
ROUGE-L   : 0.7123
TER       : 39.0244
COMET     : 0.8675

Time to 1st tok : 469.0941s
Total inf time  : 469.1039s
Tokens/sec      : 0.14
RAM   : 18.19 MB
CPU   : 0.90 %
Model size : 4728.20 MB

=== Estonian → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : I apply for a tourist visa to visit the United States. I plan to stay for two weeks and visit famous sights such as the Statue of Liberty and Times Square. After the holiday I will return to my home country.

BLEU      : 0.6359
BERTScore : 0.9786
METEOR    : 0.8266
ROUGE-L   : 0.7805
TER       : 31.7073
COMET     : 0.9179

Time to 1st tok : 562.8894s
Total inf time  : 562.9028s
Tokens/sec      : 0.11
RAM   : 17.24 MB
CPU   : 1.80 %
Model size : 4728.20 MB
